In [352]:
import numpy as np 
import pandas as pd 
import nltk 
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm 

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [353]:
stopwords = stopwords.words('english')

In [354]:
lemmatizer = WordNetLemmatizer()

In [355]:
data = pd.read_csv(r'train.csv' , encoding = 'latin1')

In [356]:
data = data.iloc[:50000 , :]

In [357]:
data.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [358]:
data = data.loc[: , ["text" , "sentiment"]]

In [359]:
data = data.dropna()

In [360]:
X , y = data["text"] , data["sentiment"]

In [361]:
def process_data(sentence):

    sentence = sentence.strip()
    lower = sentence.lower()
    words = nltk.word_tokenize(lower)

    words = [word for word in words if word not in stopwords]
    lemm_words = [lemmatizer.lemmatize(word) for word in words]

    return " ".join(lemm_words)

In [362]:
X = X.apply(process_data)

In [363]:
y_label = pd.get_dummies(y , drop_first=True )

In [364]:
X_train , X_test , y_train , y_test = train_test_split(X,  y , test_size=0.25 , shuffle=True)

In [365]:
cv = CountVectorizer()
tfidf = TfidfVectorizer()

In [366]:
x_cf_train = cv.fit_transform(X_train)
x_tf_train = tfidf.fit_transform(X_train)

In [367]:
x_cf_test  = cv.transform(X_test)
x_tf_test = tfidf.transform(X_test)

In [368]:
word2vec_model = Word2Vec(sentences=X_train, vector_size=100, window=5, min_count=1, workers=4)
cbow = Word2Vec(sentences=X_train, vector_size=100, window=5, min_count=1, workers=4 , sg = 0)
skipgram = Word2Vec(sentences=X_train, vector_size=100, window=5, min_count=1, workers=4 , sg = 1)

In [369]:
def calculate_mean_embedding(tokens , model):
    embeddings = [model.wv[token] for token in tokens if token in model.wv]
    if embeddings:
        return np.mean(embeddings, axis=0).astype(np.float32)
    else:
        return np.zeros(model.vector_size).astype(np.float32)

In [370]:
tqdm.pandas(desc="Calculating embeddings")
x_w2v_train = X_train.apply((lambda x: calculate_mean_embedding(X_train, word2vec_model)))

In [371]:
tqdm.pandas(desc="Calculating embeddings")
x_cbow_train = X_train.apply((lambda x: calculate_mean_embedding(X_train, cbow)))

In [372]:
tqdm.pandas(desc="Calculating embeddings")
x_sg_train = X_train.apply((lambda x: calculate_mean_embedding(X_train, skipgram)))

In [373]:
x_w2v_test = X_train.apply((lambda x: calculate_mean_embedding(X_test, word2vec_model)))
x_cbow_test = X_train.apply((lambda x: calculate_mean_embedding(X_test, cbow)))
x_sg_test = X_train.apply((lambda x: calculate_mean_embedding(X_test, skipgram)))

In [374]:
model_cv = LogisticRegression()
model_tf = LogisticRegression()
model_w2v = LogisticRegression()
model_cbow = LogisticRegression()
model_sg = LogisticRegression()

In [375]:
x_w2v_train = [np.array(i) for i in x_w2v_train]
x_cbow_train = [np.array(i) for i in x_cbow_train]
x_sg_train = [np.array(i) for i in x_sg_train]


x_w2v_test = [np.array(i) for i in x_w2v_test]
x_cbow_test = [np.array(i) for i in x_cbow_test]
x_sg_test = [np.array(i) for i in x_sg_test]

In [376]:
model_cv.fit(x_cf_train , y_train)
model_tf.fit(x_tf_train , y_train)
model_w2v.fit(x_w2v_train , y_train)
model_cbow.fit(x_cbow_train , y_train)
model_sg.fit(x_sg_train , y_train)

LogisticRegression()